In [1]:
import telebot
from telebot import types
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2 as ps

In [2]:
token = "7639010467:AAG4D-qM36c4MPqlPKkCM_I6ACZNEHqZ9Os"
bot=telebot.TeleBot(token)

In [3]:
conn2 = ps.connect(dbname = "postgres",#park_BD#postgres
                    user = "postgres",
                    password = "1",
                    host = "localhost",
                    port = "5432")
#connection.autocommit = True

In [ ]:
###!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!###

In [ ]:
###!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!###

In [4]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    cursor.execute(query)
    result = cursor.fetchall()
    df = pd.DataFrame(result)
    return result,df


In [5]:
def insert(connection,chat_id):
    connection.autocommit = True
    cursor = connection.cursor()

    select_users = """SELECT * FROM table_1"""
    cursor.execute(select_users)
    results = cursor.fetchall()
    
    df = pd.DataFrame(results)
    df.columns = ['ID','Chat_ID','Onlines','Zapros_Foto']
    #return df
    count_banana_all = (df['Chat_ID'] == chat_id).sum()
    if count_banana_all > 0:
        update_users = (f"""UPDATE table_1 SET "Onlines" = True WHERE "Chat_ID" ={chat_id};""")
        cursor.execute(update_users)
    else:
        id_num = df['Chat_ID'].count() + 1
        insert_query = (f"""INSERT INTO public.table_1("ID", "Chat_ID", "Onlines", "Zapros_Foto")VALUES ({id_num}, {chat_id}, True, False);""")
        cursor.execute(insert_query)

In [6]:
def stoped(connection,chat_id):
    connection.autocommit = True
    cursor = connection.cursor()

    update_users = (f"""UPDATE table_1 SET "Onlines" = False WHERE "Chat_ID" ={chat_id};""")
    cursor.execute(update_users)

In [7]:
def foto(connection,chat_id):
    connection.autocommit = True
    cursor = connection.cursor()

    update_users = (f"""UPDATE table_1 SET "Zapros_Foto" = True WHERE "Chat_ID" ={chat_id};""")
    cursor.execute(update_users)    

In [8]:
@bot.message_handler(commands=['start'])
def send_welcome(message):
    chat_id = message.from_user.id
    insert(conn2,chat_id)
    bot.send_message(message.chat.id, f'Рады видеть в нашем боте :) {chat_id}')

@bot.message_handler(commands=['stop'])
def send_welcome(message):
    chat_id = message.from_user.id
    stoped(conn2,chat_id)
    bot.send_message(message.chat.id, f'Спасибо что воспользовались нашим ботом. Пока:( {chat_id}')

@bot.message_handler(commands=['foto'])
def send_welcome(message):
    chat_id = message.from_user.id
    foto(conn2,chat_id)
    bot.send_message(message.chat.id, f'Вот твоё фото :) {chat_id}')

In [ ]:
bot.polling(none_stop = True)